Retrieve experiment results

In [2]:
import json
from os.path import join
import os

import pandas as pd

from cogspaces.pipeline import get_output_dir

from json import JSONDecodeError

basedir_ids = [1]
basedirs = [join(get_output_dir(), 'nips_multinomial', str(_id), 'run') for _id in basedir_ids]
res_list = []
for basedir in basedirs:
    for exp_dir in os.listdir(basedir):
        exp_dir = join(basedir, exp_dir)
        try:
            config = json.load(open(join(exp_dir, 'config.json'), 'r'))
            info = json.load(open(join(exp_dir, 'info.json'), 'r'))
        except (JSONDecodeError, FileNotFoundError):
            continue
        datasets = config['datasets']
        datasets = '__'.join(datasets)
        seed = config['seed']
        model = config['model']
        source = config['source']
        score = info['score']
        rank = info['rank']
        alpha = config['alpha']
        n_components = config['n_components']
        input_dropout_rate = config['input_dropout_rate']
        normalized = config['with_std']
        res = {'datasets': datasets, 'n_components': n_components, 'input_dropout_rate': input_dropout_rate, 'seed': seed, 'rank': rank,
               'source': source, 'normalized': normalized, 'alpha': alpha,
              'model': model}
        for key, value in score.items():
            res[key] = value
        if basedir == join(get_output_dir(), 'predict_multi', '11', 'run') and model == 'logistic':
            continue
        res_list.append(res)
res = pd.DataFrame(res_list)

Compare results with trace norm, with and without transfer

In [4]:
df_agg

rank  \
                                                                        mean   
normalized source        model      datasets   alpha input_dropout_rate        
True       hcp_rs        non_convex archi      0     0.0                  30   
                                                     0.1                  30   
                                                     0.2                  30   
                                                     0.3                  30   
                                                     0.4                  30   
                                                     0.5                  30   
                                    brainomics 0     0.0                  18   
                                                     0.1                  18   
                                                     0.2                  18   
                                                     0.3                  18   
                                                     0.4                  18   
                                                     0.5                  18   
                                    camcan     0     0.0                   5   
                                                     0.1                   5   
                                                     0.2                   5   
                                                     0.3                   5   
                                                     0.4                   5   
                                                     0.5                   5   
                                    la5c       0     0.0                  24   
                                                     0.1                  24   
                                                     0.2                  24   
                                                     0.3                  24   
                                                     0.4                  24   
                                                     0.5                  24   
           hcp_rs_concat non_convex archi      0     0.0                  30   
                                                     0.1                  30   
                                                     0.2                  30   
                                                     0.3                  30   
                                                     0.4                  30   
                                                     0.5                  30   
                                    brainomics 0     0.0                  18   
                                                     0.1                  18   
                                                     0.2                  18   
                                                     0.3                  18   
                                                     0.4                  18   
                                                     0.5                  18   
                                    camcan     0     0.0                   5   
                                                     0.1                   5   
                                                     0.2                   5   
                                                     0.3                   5   
                                                     0.4                   5   
                                                     0.5                   5   
                                    la5c       0     0.0                  24   
                                                     0.1                  24   
                                                     0.2                  24   
                                                     0.3                  24   
                                                     0.4                  24   
                                                     0.5                

In [5]:
df_agg = res.groupby(by=['normalized', 'source', 'model', 'datasets', 'alpha', 'input_dropout_rate']).aggregate(['mean', 'std'])

df_agg = df_agg.fillna(0)


dfs = {}
for dataset in ['brainomics', 'archi', 'camcan', 'la5c']:
    this_df = df_agg.query("datasets == '%s'" % dataset)
    indices = this_df[('test_%s' % dataset, 'mean')].groupby(level=['normalized', 'source', 'model', 'datasets']).aggregate('idxmax')
    this_df = this_df.loc[indices.values]
    dfs[dataset] = this_df

Compare results with 3 datasets

In [6]:
result_df = pd.concat(dfs.values(), keys=dfs.keys(), names=['target'])

In [8]:
result_df['test_la5c']

mean  \
target     normalized source        model      datasets   alpha input_dropout_rate             
brainomics True       hcp_rs        non_convex brainomics 0     0.5                 0.000000   
                      hcp_rs_concat non_convex brainomics 0     0.5                 0.000000   
archi      True       hcp_rs        non_convex archi      0     0.5                 0.000000   
                      hcp_rs_concat non_convex archi      0     0.5                 0.000000   
camcan     True       hcp_rs        non_convex camcan     0     0.5                 0.000000   
                      hcp_rs_concat non_convex camcan     0     0.5                 0.000000   
la5c       True       hcp_rs        non_convex la5c       0     0.5                 0.581214   
                      hcp_rs_concat non_convex la5c       0     0.5                 0.578511   

                                                                                         std  
target     normalized source        model      datasets   alpha input_dropout_rate            
brainomics True       hcp_rs        non_convex brainomics 0     0.5                 0.000000  
                      hcp_rs_concat non_convex brainomics 0     0.5                 0.000000  
archi      True       hcp_rs        non_convex archi      0     0.5                 0.000000  
                      hcp_rs_concat non_convex archi      0     0.5                 0.000000  
camcan     True       hcp_rs        non_convex camcan     0     0.5                 0.000000  
                      hcp_rs_concat non_convex camcan     0     0.5                 0.000000  
la5c       True       hcp_rs        non_convex la5c       0     0.5                 0.010219  
                      hcp_rs_concat non_convex la5c       0     0.5                 0.010430

In [10]:
df_agg

rank  \
                                                                        mean   
normalized source        model      datasets   alpha input_dropout_rate        
True       hcp_rs        non_convex archi      0     0.0                  30   
                                                     0.1                  30   
                                                     0.2                  30   
                                                     0.3                  30   
                                                     0.4                  30   
                                                     0.5                  30   
                                    brainomics 0     0.0                  18   
                                                     0.1                  18   
                                                     0.2                  18   
                                                     0.3                  18   
                                                     0.4                  18   
                                                     0.5                  18   
                                    camcan     0     0.0                   5   
                                                     0.1                   5   
                                                     0.2                   5   
                                                     0.3                   5   
                                                     0.4                   5   
                                                     0.5                   5   
                                    la5c       0     0.0                  24   
                                                     0.1                  24   
                                                     0.2                  24   
                                                     0.3                  24   
                                                     0.4                  24   
                                                     0.5                  24   
           hcp_rs_concat non_convex archi      0     0.0                  30   
                                                     0.1                  30   
                                                     0.2                  30   
                                                     0.3                  30   
                                                     0.4                  30   
                                                     0.5                  30   
                                    brainomics 0     0.0                  18   
                                                     0.1                  18   
                                                     0.2                  18   
                                                     0.3                  18   
                                                     0.4                  18   
                                                     0.5                  18   
                                    camcan     0     0.0                   5   
                                                     0.1                   5   
                                                     0.2                   5   
                                                     0.3                   5   
                                                     0.4                   5   
                                                     0.5                   5   
                                    la5c       0     0.0                  24   
                                                     0.1                  24   
                                                     0.2                  24   
                                                     0.3                  24   
                                                     0.4                  24   
                                                     0.5                

In [14]:
result_df['test_la5c']

mean  \
target     normalized source        model      datasets   alpha input_dropout_rate             
brainomics True       hcp_rs        non_convex brainomics 0     0.5                 0.000000   
                      hcp_rs_concat non_convex brainomics 0     0.5                 0.000000   
archi      True       hcp_rs        non_convex archi      0     0.5                 0.000000   
                      hcp_rs_concat non_convex archi      0     0.5                 0.000000   
camcan     True       hcp_rs        non_convex camcan     0     0.5                 0.000000   
                      hcp_rs_concat non_convex camcan     0     0.5                 0.000000   
la5c       True       hcp_rs        non_convex la5c       0     0.5                 0.581214   
                      hcp_rs_concat non_convex la5c       0     0.5                 0.578511   

                                                                                         std  
target     normalized source        model      datasets   alpha input_dropout_rate            
brainomics True       hcp_rs        non_convex brainomics 0     0.5                 0.000000  
                      hcp_rs_concat non_convex brainomics 0     0.5                 0.000000  
archi      True       hcp_rs        non_convex archi      0     0.5                 0.000000  
                      hcp_rs_concat non_convex archi      0     0.5                 0.000000  
camcan     True       hcp_rs        non_convex camcan     0     0.5                 0.000000  
                      hcp_rs_concat non_convex camcan     0     0.5                 0.000000  
la5c       True       hcp_rs        non_convex la5c       0     0.5                 0.010219  
                      hcp_rs_concat non_convex la5c       0     0.5                 0.010430

In [43]:
df_agg.query("datasets == 'brainomics__archi__hcp'")

rank  \
                                                                                                                 mean   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate        
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.80                0.25                 71   
                                                                                              0.50                 71   
                                                                          0.90                0.25                 71   
                                                                                              0.50                 71   
                                                                          0.95                0.25                 71   
                                                                                              0.50                 71   

                                                                                                                       \
                                                                                                                  std   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate        
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.80                0.25                0.0   
                                                                                              0.50                0.0   
                                                                          0.90                0.25                0.0   
                                                                                              0.50                0.0   
                                                                          0.95                0.25                0.0   
                                                                                              0.50                0.0   

                                                                                                                        seed  \
                                                                                                                        mean   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate               
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.80                0.25                2330081002   
                                                                                              0.50                2330081002   
                                                                          0.90                0.25                2330081002   
                                                                                              0.50                2330081002   
                                                                          0.95                0.25                2330081002   
                                                                                              0.50                2330081002   

                                                                                                                                \
                                                                                                                           std   
normalized source          model      datasets               n_components latent_dropout_rate input_dropout_rate                 
True       hcp_rs_positive non_convex brainomics__archi__hcp 200          0.80                0.25                1.242420e+09   
                                                                                              0.50                1.242420e+09   
                                                                          0.90                0.25                1.242420e+09   


In [44]:
df

NameError: name 'df' is not defined